## Thu thập dữ liệu gồm OHLC data và corporate action data
- Quá khứ đến 1.10.2025
- Tất cả nếu lấy sau ngày 1.10 thì tuyệt đối không đụng đến để đoạn từ 1.10 đến
31.12 sẽ bung ra sau (giả thực tế)
## Xử lý dữ liệu
- Stock Split
- Dividends
- Missing values & outliers
## Technical Indicators and Build Signal
- Stock Return (Log Returns)
- Build Momentum Signal (tín hiệu có thể là vị trí của giá so với Simple Moving Average (SMA)
hoặc Bollinger Bands)
## Cross-sectional
- Ranking
- Long/Short Position (Mua/Bán n cổ phiếu có thứ hạng cao đến thấp/thấp đến cao, Equal Weighting)
- Rebalancing (Giữ danh mục đầu tư này trong 1 tháng. Vào cuối tháng, đóng tất cả các vị thế, tính toán lại signal,
ranking lại, và mở các vị thế long/short mới cho tháng tiếp theo.)
## Backtesting
- Tính lợi nhuận hàng tháng của danh mục. Lợi nhuận của danh mục là trung bình của lợi nhuận từ các vị thế long và short.
- Statistical Analysis (Hypothesis Testing, "Alpha" - lợi nhuận vượt trội so với thị trường chuẩn, sử dụng QQ Plot để kiểm tra xem nó có "đuôi béo" (fat tails) hay không - cho thấy rủi ro cao hơn so với phân phối chuẩn.)
- Regression (tránh Bias)

### Lưu ý
Lúc làm data lấy từ 1.2015 đến 31.9.2025
Pick random 80:20/ 70:30
6 tháng / 3 tháng
6.2016 - 12.2016, 1.2018-6.2018, 3.2021-12.2021
